### Main notebook

In [1]:
import pandas as pd
import numpy as np
import json

import networkx as nx

from tqdm import tqdm

from compound import Compound
from reaction import Reaction
from graph import Graph
from data import Data

# read data from csv
cpds = pd.read_csv('../GNN_toxic/data/raw/compounds_final.csv', index_col=0) # containing toxicity
rxns = pd.read_csv('data/reactions_final.csv', index_col=0)
pairs = pd.read_csv('data/pairs_final.csv', index_col=0)
cofactors = pd.read_csv('data/original/cofactors_KEGG.csv')

# create class instances
data = Data()
graph = Graph(pairs=pairs)

In [2]:
# Create a Compound object for each row in the DataFrame and add it to the data
for index, row in cpds.iterrows():
    entry = row['Entry']
    name = row['Names']
    formula = row['Formula']
    mw = row['mol_weight']
    smiles = row['SMILES']
    is_cofactor = row['Entry'] in cofactors['Entry'].values
    is_toxic = row['toxic']

    compound = Compound(entry, name, formula, mw, smiles, is_cofactor, is_toxic)
    data.add_element('compound', compound)

# Create a Reaction object for each row in the DataFrame and add it to the data
for index, row in rxns.iterrows():
    entry = row['Entry']
    name = row['Names']
    compounds = row['Compound']
    enzyme = row['EC Number']

    reaction = Reaction(entry, name, compounds, enzyme)
    data.add_element('reaction', reaction)

In [3]:
# number of times a metabolite apperas on pairs dataset
graph.get_number_of_occurences(pairs)
graph.create_graph(data=data, pairs=pairs)

# nodes: 8591 
# edges: 30026


100%|██████████| 8591/8591 [00:00<00:00, 666455.80it/s]


In [4]:
graph.calculate_edge_mol_weight(data)
# graph.calculate_smiles_similarity(data)

100%|██████████| 30026/30026 [00:00<00:00, 635877.95it/s]


In [5]:
######### VALIDATION SET FROM nicepath ###########
test_cases = pd.read_csv('data/original/test_cases.csv')
test_cases['source'] = test_cases['Pathway '].apply(lambda x: x.split(',')[0])
test_cases['target'] = test_cases['Pathway '].apply(lambda x: x.split(',')[len(x.split(','))-1])
test_cases['paths_list'] = test_cases['Pathway '].apply(lambda x: x.split(','))
paths = graph.validate(test_cases, 'mol_weight')

Correct pathway predictions: 14
Correct pathway predictions (%): 28.0


In [6]:
1/0

ZeroDivisionError: division by zero

In [16]:
for i in graph.shortest_simple_paths('C00082', 'C01533', length=10, weight='mol_weight'):
    print(i)

['C00082', 'C00355', 'C10447', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00822', 'C00355', 'C10447', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00822', 'C05604', 'C00355', 'C10447', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00811', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00811', 'C16828', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00355', 'C04045', 'C22038', 'C10447', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00822', 'C00355', 'C04045', 'C22038', 'C10447', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00822', 'C05604', 'C00355', 'C04045', 'C22038', 'C10447', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00822', 'C01693', 'C05604', 'C00355', 'C10447', 'C01197', 'C01494', '

In [12]:
test_cases

,Pathway,source,target,paths_list
0,"C00082,C00811,C01197,C01494,C05619,C00482,C056...",C00082,C01533,"[C00082, C00811, C01197, C01494, C05619, C0048..."
1,"C00223,C00323,C00406,C02666,C00590,C00761",C00223,C00761,"[C00223, C00323, C00406, C02666, C00590, C00761]"
2,"C00811,C01197,C01494,C05619,C00482,C01175,C02887",C00811,C02887,"[C00811, C01197, C01494, C05619, C00482, C0117..."
3,"C00079,C00423,C00540,C00903",C00079,C00903,"[C00079, C00423, C00540, C00903]"
4,"C00223,C00323,C00406,C17741,C10443,C17749,C17750",C00223,C17750,"[C00223, C00323, C00406, C17741, C10443, C1774..."
5,"C06561,C00509,C16492,C01460",C06561,C01460,"[C06561, C00509, C16492, C01460]"
6,"C05903,C12249,C12634,C12635,C12636",C05903,C12636,"[C05903, C12249, C12634, C12635, C12636]"
7,"C01477,C01514,C03515,C12632,C04900",C01477,C04900,"[C01477, C01514, C03515, C12632, C04900]"
8,"C05903,C00389,C10107,C12633,C11620",C05903,C11620,"[C05903, C00389, C10107, C12633, C11620]"
9,"C05905,C16298,C08639,C12096,C16299",C05905,C16299,"[C05905, C16298, C08639, C12096, C16299]"


### Try cluster graph nodes

### Try similarity based on SMILES

In [ ]:
smiles1 = data.get_compound_by_id('C00223').smiles
smiles2 = data.get_compound_by_id('C00323').smiles

from rdkit import Chem
from rdkit import DataStructs

ms = [Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)]
fs = [Chem.RDKFingerprint(x) for x in ms]
s = DataStructs.FingerprintSimilarity(fs[0], fs[1])
print('Similarity: '+str(s))


smiles1 = data.get_compound_by_id('C00223').smiles
smiles2 = data.get_compound_by_id('C12096').smiles

from rdkit import Chem
from rdkit import DataStructs

ms = [Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)]
fs = [Chem.RDKFingerprint(x) for x in ms]
s = DataStructs.FingerprintSimilarity(fs[0], fs[1])
print('Similarity: '+str(s))

Similarity: 0.989145183175034
Similarity: 0.5102379634753735
